In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
data2001 = pd.read_csv('./pems05/d05_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

,Timestamp,Station,District,Freeway #,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,Avg Occupancy,Avg Speed
0,02/28/2018 00:00:00,500010011,5,1,N,ML,2.713,9,100,0.0,0.0000,65.0
1,02/28/2018 00:00:00,500010012,5,1,S,ML,2.714,9,100,1.0,0.0011,65.0
2,02/28/2018 00:00:00,500010021,5,1,N,ML,0.697,0,0,11.0,0.0103,67.6
3,02/28/2018 00:00:00,500010022,5,1,S,ML,0.698,0,0,11.0,0.0103,67.6
4,02/28/2018 00:00:00,500010031,5,1,N,ML,0.831,0,0,40.0,0.0235,64.1
...,...,...,...,...,...,...,...,...,...,...,...,...
69979,02/28/2018 23:55:00,501560032,5,156,E,ML,4.179,0,0,26.0,0.0122,67.7
69980,02/28/2018 23:55:00,501560061,5,156,W,ML,4.730,7,100,3.0,0.0027,65.0
69981,02/28/2018 23:55:00,501560062,5,156,E,ML,4.729,8,100,6.0,0.0061,64.9
69982,02/28/2018 23:55:00,501560081,5,156,E,ML,4.729,9,100,1.0,0.0012,64.8


In [4]:
comm = np.load('pems05_comm.npy')
comm

array([500010092, 500010093, 500010101, 500010102, 500010121, 500010122,
       500010142, 500010143, 500011021, 500011022, 500011032, 500011033,
       500011052, 500011053, 500011062, 500011063, 500011072, 500011073,
       500011092, 500011102, 500011121, 500011123, 500011141, 500011143,
       500014091, 500014092, 500014101, 500014102, 500014111, 500014112,
       500014121, 500014122, 500014132, 500014133, 500017011, 500017012,
       501010022, 501010023, 501010031, 501010032, 501010052, 501010053,
       501010061, 501010062, 501010102, 501010103, 501010112, 501010113,
       501010131, 501010132, 501010152, 501010153, 501011032, 501011041,
       501011091, 501011092, 501011121, 501011122, 501011131, 501011132,
       501011141, 501011142, 501011151, 501012021, 501012022, 501012042,
       501012043, 501012071, 501012072, 501012081, 501012082, 501012101,
       501012102, 501012111, 501012112, 501016071, 501016072, 501016081,
       501016082, 501016091, 501016092, 501016101, 

In [5]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    # Read the CSV file, specifying the usecols parameter to only load the columns of interest
    data = pd.read_csv(
        file_path, 
        header=None, 
        usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
        names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
        compression='gzip'
    )

    # Convert the Timestamp column to datetime format and set it as the index
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data.set_index('Timestamp', inplace=True)

    # Pivot the table to get Stations as columns and Total Flow as cell values
    pivot_data = data.pivot(columns='Station', values='Avg Flow')

    return pivot_data

# Read the data for February 28 to get the column names (Station IDs)
# feb_28_data = read_and_process_data('./pems03/d03_text_station_5min_2018_02_28.txt.gz')
# feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2012, 3, 7)  # 开始日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems05/d05_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
    
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems05_all_common_flow.csv')

In [ ]:
combined_data

In [1]:
import pandas as pd
pems05 = pd.read_csv('pems05_all_common_flow.csv')

# Convert the 'date' column to datetime
pems05['date'] = pd.to_datetime(pems05['date'])

# Set the 'date' column as the index
pems05.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems05_hourly = pems05.resample('H').sum()

# Reset the index if you want the 'date' column back
pems05_hourly.reset_index(inplace=True)
pems05_hourly.to_csv('pems05_h.csv',index=False)


# Convert the 'date' column to datetime
pems05 = pd.read_csv('pems05_all_common_flow.csv')
pems05['date'] = pd.to_datetime(pems05['date'])

# Set the 'date' column as the index
pems05.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems05_daily = pems05.resample('D').sum()

# Reset the index if you want the 'date' column back
pems05_daily.reset_index(inplace=True)
pems05_daily.to_csv('pems05_d.csv',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_45504\2035277446.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pems05_hourly = pems05.resample('H').sum()
